In [1]:
import networkx as nx
from collections import defaultdict
import random
import math
from karateclub import DeepWalk

/home/chaty/Desktop/BTP/Sem-7/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
# Values of hyper parameters as mentioned in the research paper

alpha = 0.7    # Used in probability calculation
beta = 0.3    # Used in probability calculation

walk_length = 10    # The length of path in deepwalk
dim = 2    # The dimension in Deepwalk
window_size = 4    # Window size in Deepwalk

Nc = 50    # Maximum number of cycles/iterations
L = 5    # Length of ant selection path
m = 30    # Number of ants

rho = 0.05    # Pheromone evaporation coefﬁcient
budget = 500
negative_influence_set_size = 10

In [3]:
def get_heuristic_value_for_all_edges(graph: nx.graph):
    
    model = DeepWalk(walk_length = walk_length, dimensions = dim, window_size = window_size)
    model.fit(graph)
    embedding = model.get_embedding()


    heuristic_value = {}
    for u, v in graph.edges:
        if u == v:
            continue
            
        xu, yu = embedding[u]
        xv, yv = embedding[v]

        x = xu - xv
        y = yu - yv
        heuristic_value[(u, v)] = math.sqrt((x ** 2) + (y ** 2))
    
    mn = min(heuristic_value.values())
    mx = max(heuristic_value.values())
    range_ = mx - mn

    for key, value in heuristic_value.items():
        h = (value - mn) / range_
        u, v = key
        graph.add_edge(u, v, heuristic_value = h)

In [4]:
def generate_ant(graph: nx.graph, negative_influence_set: set, path_size: int):
    node = random.choice(list(negative_influence_set))

    selected = set()
    selected.add(node)
    path = [node]

    for _ in range(path_size - 1):
        neighbours = [u for u in graph[node] if u not in selected]
        if len(neighbours) == 0:
            break

        attractiveness = []
        for u in neighbours:
            tau = graph[node][u]['pheromone']
            eta = graph[node][u]['heuristic_value']

            attractiveness.append((tau ** alpha) * (eta ** beta))
        
        random_value = random.random()
        total = sum(attractiveness)
        probabilities = [value/total for value in attractiveness]
        
        node_index = -1
        total_prob = 0

        for index in range(len(probabilities)):
            total_prob += probabilities[index]
            if total_prob >= random_value:
                node_index = index
                break
        
        node = neighbours[node_index]
        selected.add(node)
        path.append(node)
    
    return path

In [5]:
# Returns first order neighbours of node in a graph
def first_order_degree(graph: nx.graph, node: int):
    return len(graph.adj[node])

# Returns second order neighbours of node in a graph
def second_order_degree(graph: nx.graph, node: int):
    visited = set(graph[node])
    visited.add(node)

    degree = 0
    for u in graph[node]:
        for v in graph[u]:
            if v not in visited:
                degree = degree + 1
                visited.add(v)

    return degree

i = 1    # Some coefficient, not mentioned properly
j = 1    # Some coefficient, not mentioned properly

def fitness(graph: nx.graph, ant: list):
    fitness = 0
    for u in ant:
        fitness = fitness + (first_order_degree(graph, u) ** i) * (second_order_degree(graph, u) ** j)

    return fitness

In [6]:
def update_pheromones(graph: nx.graph, population: list, evaporation_coefficient: float):
    fitness_along_edge = {}

    for ant_path in population:
        fitness_ = fitness(graph, ant_path)
        for u, v in zip(ant_path, ant_path[1:]):
                fitness_along_edge[(u, v)] = fitness_along_edge.get((u, v), 0) + fitness_
    
    for u, v in graph.edges:
        if u == v:
            continue
        
        graph[u][v]['pheromone'] = \
            (1 - evaporation_coefficient) * graph[u][v]['pheromone'] + fitness_along_edge.get((u, v), 0)

In [7]:
graph = nx.DiGraph()
no_of_nodes = 50
no_of_edges = 250

# Constructing the graph
for i in range(no_of_nodes):
    graph.add_node(i)

included = set()

cal = 0
for i in range(no_of_edges):
    u = random.randint(0, no_of_nodes - 1)
    v = random.randint(0, no_of_nodes - 1)

    while u == v or (u, v) in included:
        u = random.randint(0, no_of_nodes - 1)
        v = random.randint(0, no_of_nodes - 1)

    graph.add_edge(u, v, pheromone = random.random())
    included.add((u, v))

    cal += 1

In [8]:
for u in graph.nodes():
    graph.nodes[u]['cost'] = random.randint(1, 200)

In [9]:
negative_influence_set = set()

for i in range(negative_influence_set_size):
    u = random.choice(list(graph.nodes()))

    while u in negative_influence_set:
        u = random.choice(list(graph.nodes()))
    
    negative_influence_set.add(u)

In [42]:
def ACO_GE(graph: nx.graph, no_of_iterations: int, no_of_ants: int, negative_influence_set: set, 
           path_size: int, evaporation_coefficient: float, budget: int):

    get_heuristic_value_for_all_edges(graph)
    
    population = []

    for i in range(no_of_iterations):
        population.clear()

        for k in range(no_of_ants):
            ant = generate_ant(graph, negative_influence_set, path_size)
            population.append(ant)

        
        update_pheromones(graph, population, evaporation_coefficient)
    
    candidate_set = set()
    for ant in population:
        for u in ant:
            candidate_set.add(u)

    candidate_set = list(candidate_set)
    candidate_set.sort(key = lambda x: graph.nodes[x]['cost'], reverse = True)

    seed_set = set()
    while len(candidate_set) and budget >= graph.nodes[candidate_set[-1]]['cost']:
        u = candidate_set[-1]

        budget -= graph.nodes[u]['cost']

        seed_set.add(u)
        candidate_set.pop()
    
    return seed_set

positive_set = ACO_GE(graph, Nc, m, negative_influence_set, L, rho, budget)

print(positive_set)

{32, 6, 9, 44, 45, 16, 17, 18, 49, 22, 29, 31}
